# Phase 4 - Dashboard Interactif (Dash)

## 4.1 Objectifs Dashboard

*   Exploration dynamique du marché de l'emploi
*   Filtres métier / secteur / localisation
*   Visualisation des clusters ML
*   Lecture business (salaires, types de contrats)

## 4.2 Structure du projet

```
project/
|-- app.py
|-- hellowork_ml_enriched.csv
```

## 4.3 Application Dash (app.py)

Le code ci-dessous définit l'application Dash. Vous pouvez l'exécuter directement dans ce notebook (attention, cela bloquera le noyau jusqu'à l'arrêt manuel) ou le copier dans un fichier `app.py`.

In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Initialisation de l'application
app = dash.Dash(__name__)

# Chargement des données enrichies
df = pd.read_csv("hellowork_ml_enriched.csv")
if 'Publication_Date' in df.columns:
    df['Publication_Date'] = pd.to_datetime(df['Publication_Date'])

# Layout de l'application
app.layout = html.Div([
    html.H1("Analyse du Marché de l'Emploi - Hellowork"),
    
    dcc.Dropdown(
        id='sector_filter',
        options=[{'label': s, 'value': s} for s in df['Sector'].unique()],
        multi=True,
        placeholder="Filtrer par secteur"
    ),
    
    html.Div([
        dcc.Graph(id='cluster_chart', style={'width': '49%', 'display': 'inline-block'}),
        dcc.Graph(id='salary_chart', style={'width': '49%', 'display': 'inline-block'}),
    ]),

    html.H2("Analyse Temporelle (Tendances/Semaine)"),
    dcc.Graph(id='trend_chart')
])

# Callback pour la mise à jour des graphiques
@app.callback(
    [Output('cluster_chart', 'figure'), Output('salary_chart', 'figure'), Output('trend_chart', 'figure')],
    [Input('sector_filter', 'value')]
)
def update_charts(sectors):
    dff = df if not sectors else df[df['Sector'].isin(sectors)]
    
    # Graphique 1 : Clusters Métiers
    fig1 = px.histogram(dff, x='Job_Cluster', 
                        title='Répartition des clusters métiers')
    
    # Graphique 2 : Classification Salariale
    fig2 = px.histogram(dff, x='Salary_Class_Pred', 
                        title='Classification salariale')

    # Graphique 3 : Tendance Temporelle
    if 'Publication_Date' in dff.columns:
        df_trend = dff.groupby(pd.Grouper(key='Publication_Date', freq='W')).size().reset_index(name='Count')
        fig3 = px.line(df_trend, x='Publication_Date', y='Count', 
                       title="Nombre d'offres publiées par semaine", markers=True)
    else:
        fig3 = px.histogram(x=[], title="Date de publication manquante")
    
    return fig1, fig2, fig3

# Lancement du serveur
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False) # use_reloader=False pour éviter les doublons dans Jupyter

## Valeur académique & BI

*   Pipeline complet **Scraping -> ETL -> ML -> Dashboard**
*   ML utilisé comme **outil d'aide à l'analyse**, pas boîte noire
*   Résultats interprétables et présentables en soutenance
*   Projet aligné avec les standards Business Intelligence modernes